# Automatic Differentiation


In [41]:
import torch

Calculations for simple derivatives are straightforward, for complex models, working out the updates by hand is **unfeasible**.

Deep learning frameworks expedite this work
by automatically calculating derivatives, i.e., *automatic differentiation*.

Based on our designed model the system builds an acyclic *computational graph*, tracking which data combined through which operations to produce the output.

Automatic differentiation enables the system to subsequently backpropagate gradients.
Here, *backpropagate* simply means to trace through the computational graph,
filling in the partial derivatives with respect to each parameter.

**Example**: differentiating the function
$y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to the column vector $\mathbf{x}$.

**What is the derivative of this function ?**

In [42]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

Before we calculate the gradient
of $y$ with respect to $\mathbf{x}$,
we need a place to store it

In [43]:
x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
x.grad  # The default value is None

We now calculate our function of `x` and assign the result to `y`

In [44]:
y = 2 * torch.dot(x.T, x)
y

tensor(28., grad_fn=<MulBackward0>)

We can now take the gradient of `y`
with respect to `x`

In [45]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

We already know that the gradient of the function $y = 2\mathbf{x}^{\top}\mathbf{x}$ with respect to $\mathbf{x}$ should be $4\mathbf{x}$

In [46]:
x.grad == 4 * x

tensor([True, True, True, True])

Now let's calculate 
another function of `x`
and take its gradient

PyTorch **accumulates the gradient by default**, we need to clear the previous values

In [47]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

Sum up the gradients
computed individually for each example

In [48]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))
x.grad

tensor([0., 2., 4., 6.])

Move some calculations
outside of the recorded computational graph

In [49]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad

tensor([0., 1., 4., 9.])

Even if
a function required passing through a maze of Python control flow
we can still calculate the gradient of the resulting variable

In [50]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(1,), requires_grad=True)
d = f(a)
d.backward()

a.grad

tensor([1024.])